In [1]:
import pandas as pd
import numpy as np
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)  
from sklearn import preprocessing, cross_validation
from sklearn.neural_network import MLPClassifier

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
def make_network(FILENAME):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    from sklearn.neural_network import MLPClassifier
    """Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and proper hot encoding is done. Other stuff too probably.
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    X_train : array
        Numpy array of the training inputs.
    y_train : array
        Numpy array of the training outputs.
    
    X_test : array
        Numpy array of the testing inputs.
    y_test : array
        Numpy array of the testing outputs.
    Sike!
    
    model : object
        MLP which can predict the outcome of NBA games
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    data = read_csv(FILENAME)
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values)
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['FT'],data['FTA']
    data = get_dummies(data)
    del data['VENUE_Home'],data['VENUE_Road']
    #print(data)
    
    dat = []

    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
            
    """At this point in the data dat is an array arranged as 
        OR  DR  TOT   A  PF  ST  TO  BL  PTS       3P%       FG%       FT% x2  (for road then home )
    so next up is to hot pull road and home points 
    
    road points is column 8 and home points is column 20. 
    
    So grab those from dat and make y. 
    
    X is every other column, so merge everyone else 
    
    """
    dat = np.array(dat)   
  #  print(dat[0])
    y = []
    for j in range(len(dat)):
        roadpts = dat[j,8]
        homepts = dat[j,20]
       # print(home,road)
    #distinguish home win or road win. 
        if homepts > roadpts:
            y.append(np.array([0,1]))  # 0 1 mean home won
        else:
            y.append(np.array([1,0]))  #  1 0 means away won. 
    y = np.array(y)
    X1 = np.concatenate((dat[:,0:8],dat[:,9:20]),axis=1)
    X = np.concatenate((X1,dat[:,21:24]),axis=1)    #need to go one further column to snag HFT% 
    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.4)
  #  print(X_train[0])
  #  print(X_test[0])

    model = MLPClassifier()
    model.batch_size = 20
    model.fit(X_train,y_train)
    
    print(model.score(X_test,y_test))
    return model
    

In [13]:
noahbets = make_network('1517_boxscores_nospreads.csv')

0.8704761904761905


In [14]:
def make_prediction_data(filename):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    data = read_csv(filename)
    #convert to per game stats and sort columns. 
    data['ORB'] =  np.divide(data['ORB'].values,data['G'].values)
    data['DRB'] =  np.divide(data['DRB'].values,data['G'].values)
    data['TRB'] =  np.divide(data['TRB'].values,data['G'].values)
    data['AST'] =  np.divide(data['AST'].values,data['G'].values)
    data['STL'] =  np.divide(data['STL'].values,data['G'].values)
    data['BLK'] =  np.divide(data['BLK'].values,data['G'].values)
    data['TOV'] =  np.divide(data['TOV'].values,data['G'].values)
    data['PF'] =  np.divide(data['PF'].values,data['G'].values)
    teams  = data['Team']

    data = data[['ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK' ,'3P%','FG%' ,'FT%']]
    print("Here is every teams index value: ")
    print(teams)
    return teams,data

In [16]:
def game_maker(roadteam,hometeam):
    """After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game =np.concatenate((roadteam,hometeam))
    return game

In [17]:
teams, stats =make_prediction_data('3-12teamstats.csv')

Here is every teams index value: 
0      Golden State Warriors
1            Houston Rockets
2     Minnesota Timberwolves
3           Toronto Raptors*
4       New Orleans Pelicans
5             Denver Nuggets
6        Cleveland Cavaliers
7      Oklahoma City Thunder
8         Washington Wizards
9         Los Angeles Lakers
10         Charlotte Hornets
11      Los Angeles Clippers
12              Phoenix Suns
13            Indiana Pacers
14             Brooklyn Nets
15             Orlando Magic
16        Philadelphia 76ers
17            Boston Celtics
18    Portland Trail Blazers
19           New York Knicks
20             Atlanta Hawks
21           Milwaukee Bucks
22                 Utah Jazz
23                Miami Heat
24          Dallas Mavericks
25             Chicago Bulls
26           Detroit Pistons
27         San Antonio Spurs
28          Sacramento Kings
29         Memphis Grizzlies
30            League Average
Name: Team, dtype: object


Ok let's do ML tonight :DK

Games: 
Rockets @ Blazers
Raptors @ Magic
Thunder @ Celtics
Clippers @ Twolves 
Mavs @ Pels
Hawks @ Jazz
Pistons @ Suns


How'd it do yesterday? 

In [18]:
game1 = game_maker(stats.values[1],stats.values[18])
game2 = game_maker(stats.values[3],stats.values[15])
game3 = game_maker(stats.values[7],stats.values[17])
game4 =  game_maker(stats.values[11],stats.values[2])
game5 = game_maker(stats.values[24],stats.values[4])
game6 = game_maker(stats.values[20],stats.values[22])
game7 = game_maker(stats.values[26],stats.values[12])


In [19]:
noahbets.predict(np.array([game1,game2,game3,game4,game5,game6,game7]))

array([[1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0]])